# Sandbox Notebook

Experimental notebook for testing `home_media` package functionality.

Auto-reload is enabled to pick up changes to the package without restarting the kernel.

In [ ]:
# Enable auto-reload for the home_media package
%load_ext autoreload
%autoreload 2

In [ ]:
# Imports
import sys
from pathlib import Path
import yaml
import pandas as pd
import datetime

# Add parent directory to path so we can import home_media
sys.path.insert(0, str(Path.cwd().parent))

## Load Configuration

In [ ]:
# Load config from config.yaml
config_path = Path.cwd().parent / "config.yaml"

with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

print("Configuration loaded:")
print(f"photos_root_original: {config['photos_root_original']}")

## List Subdirectories in photos_root_original

In [ ]:
# Get the photos root directory
photos_root = Path(config['photos_root_original'])
# photos_root = Path(r'\\tiger\home\Photos\MobileBackup\Pixel 8 Pro\DCIM\Camera')

print(f"Listing subdirectories in: {photos_root}\n")

# Check if path exists
if not photos_root.exists():
    print(f"ERROR: Path does not exist: {photos_root}")
elif not photos_root.is_dir():
    print(f"ERROR: Path is not a directory: {photos_root}")
else:
    # List all subdirectories
    subdirs = [d for d in photos_root.iterdir() if d.is_dir()]
    
    print(f"Found {len(subdirs)} subdirectories:\n")
    
    for subdir in sorted(subdirs):
        print(f"  📁 {subdir.name}")

In [ ]:
# Look at files in a specific subdirectory
subdir = "2025/01/01"
# subdir = "2025/12"
target_dir = photos_root / subdir

print(f"Examining directory: {target_dir}")
print(f"Subdirectory (relative): {subdir}")

if target_dir.exists() and target_dir.is_dir():
    file_list = [f for f in target_dir.iterdir() if f.is_file()]
    print(f"Number of files: {len(file_list)}")

    # Show first 5 files as examples
    print(f"\nFirst 5 files:")
    for f in file_list[:5]:
        print(f"  {f.name}")
else:
    print("ERROR: Directory does not exist or is not accessible")

# Create a list to hold file metadata
file_data = []

if target_dir.exists() and target_dir.is_dir():
    for file_path in target_dir.iterdir():
        if file_path.is_file():
            # Get file stats
            stats = file_path.stat()

            # Get relative subdirectory path from photos_root_original
            rel_subdir = file_path.parent.relative_to(photos_root)

            file_data.append({
                'filename': file_path.name,
                'extension': file_path.suffix.lower(),
                'subdirectory': str(rel_subdir),
                'created': datetime.datetime.fromtimestamp(stats.st_ctime),
                'modified': datetime.datetime.fromtimestamp(stats.st_mtime),
                'size_bytes': stats.st_size
            })

# Create DataFrame
df = pd.DataFrame(file_data)

print(f"\nCreated DataFrame with {len(df)} files")
print(f"\nFirst 10 rows:")
df.head(10)

## Analyze Filename Patterns

In [ ]:
# Analyze filename patterns to find common prefixes
from collections import defaultdict

# Group files by their base name (without extension)
file_groups = defaultdict(list)

if target_dir.exists() and target_dir.is_dir():
    for file_path in target_dir.iterdir():
        if file_path.is_file():
            # Get the stem (filename without extension)
            base_name = file_path.stem
            file_groups[base_name].append(file_path)

# Show some examples of grouped files
print(f"Total unique base names: {len(file_groups)}\n")
print("Examples of files with same base name:\n")

count = 0
for base_name, files in sorted(file_groups.items()):
    if len(files) > 1:  # Only show groups with multiple files
        print(f"{base_name}:")
        for f in files:
            print(f"  - {f.name}")
        print()
        count += 1
        if count >= 5:  # Show first 5 examples
            break

# Count how many base names have multiple files
multi_file_groups = {k: v for k, v in file_groups.items() if len(v) > 1}
print(f"\nBase names with multiple files: {len(multi_file_groups)}")
print(f"Base names with single file: {len(file_groups) - len(multi_file_groups)}")

## Create Grouped DataFrame (One Row Per Base Name)

In [ ]:
# Create DataFrame with one row per base name (grouped files)
grouped_data = []

for base_name, files in sorted(file_groups.items()):
    # Get all extensions for this base name
    extensions = [f.suffix.lower() for f in files]

    # Get the earliest created and latest modified dates across all variants
    created_dates = [datetime.datetime.fromtimestamp(f.stat().st_ctime) for f in files]
    modified_dates = [datetime.datetime.fromtimestamp(f.stat().st_mtime) for f in files]

    # Get total size of all variants
    total_size = sum(f.stat().st_size for f in files)

    # Get relative subdirectory (same for all files in group)
    rel_subdir = files[0].parent.relative_to(photos_root)

    grouped_data.append({
        'base_name': base_name,
        'file_count': len(files),
        'extensions': ', '.join(sorted(extensions)),
        'subdirectory': str(rel_subdir),
        'created': min(created_dates),
        'modified': max(modified_dates),
        'total_size_bytes': total_size
    })

# Create grouped DataFrame
df_grouped = pd.DataFrame(grouped_data)

print(f"Created grouped DataFrame with {len(df_grouped)} unique base names")
print(f"\nFirst 10 rows:")
df_grouped.head(10)

In [ ]:
# Create a more sophisticated grouping that handles various naming patterns
# Patterns to handle:
# 1. Standard: "2025-01-01_00-28-40.jpg" and "2025-01-01_00-28-40_001.jpg"
# 2. Google Pixel: "PXL_20251210_200246684.RAW-01.COVER.jpg" and "PXL_20251210_200246684.RAW-02.ORIGINAL.dng"
# 3. XMP sidecars: "filename.jpg.xmp"

from collections import defaultdict
import re

image_groups = defaultdict(list)

if target_dir.exists() and target_dir.is_dir():
    for file_path in target_dir.iterdir():
        if file_path.is_file():
            filename = file_path.name

            # Strategy: Find the base name by detecting common patterns
            base_name = filename

            # Pattern 1: Google Pixel RAW files (PXL_timestamp.RAW-##.TYPE.ext)
            # Extract everything before ".RAW-"
            if '.RAW-' in filename.upper():
                base_name = re.split(r'\.RAW-', filename, flags=re.IGNORECASE)[0]
            else:
                # Pattern 2: Standard files - remove all extensions first
                name_without_ext = filename
                while '.' in name_without_ext:
                    name_without_ext = Path(name_without_ext).stem

                # Pattern 3: Check for numeric suffix like _001, _002 at the end
                match = re.match(r'^(.+?)(_\d+)?$', name_without_ext)
                if match:
                    base_name = match.group(1)
                else:
                    base_name = name_without_ext

            # Calculate the suffix (everything after base_name)
            suffix = filename[len(base_name):]

            image_groups[base_name].append({
                'file_path': file_path,
                'suffix': suffix
            })

# Create DataFrame with one row per image
image_data = []

for base_name, files_info in sorted(image_groups.items()):
    # Get all suffixes
    suffixes = [info['suffix'] for info in files_info]

    # Get file paths
    file_paths = [info['file_path'] for info in files_info]

    # Get stats from all files
    created_dates = [datetime.datetime.fromtimestamp(f.stat().st_ctime) for f in file_paths]
    modified_dates = [datetime.datetime.fromtimestamp(f.stat().st_mtime) for f in file_paths]
    total_size = sum(f.stat().st_size for f in file_paths)

    # Get relative subdirectory
    rel_subdir = file_paths[0].parent.relative_to(photos_root)

    image_data.append({
        'base_name': base_name,
        'suffixes': suffixes,
        'file_count': len(suffixes),
        'subdirectory': str(rel_subdir),
        'created': min(created_dates),
        'modified': max(modified_dates),
        'total_size_bytes': total_size
    })

# Create DataFrame
df_images = pd.DataFrame(image_data)

print(f"Created image DataFrame with {len(df_images)} unique images")
print(f"\nFirst 10 rows:")
df_images.head(10)